In [1]:
#NLTK and Other Libraries
import os
import io
import re
import itertools
import operator
from collections import Counter
from langdetect import detect
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
import numpy as np
import spacy
from difflib import SequenceMatcher
import en_core_web_sm
nlp = en_core_web_sm.load()

#Data Visualization Libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.markers import MarkerStyle
import seaborn as sns

#Date time Libraries
import time
import datetime

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lingadhar_duggirala\AppData\Roaming\nltk_data
[nltk_data]     ...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
sec_metadata=pd.read_excel("C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\Sample Metadata SEC_Final.xlsx")
a=sec_metadata[(sec_metadata["Has Income Statement Flag"]==False)&(sec_metadata.PeriodType.isin(["Annual","Quarter","SemiAnnual"]))]
sec_metadata_final=sec_metadata.merge(a, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)

In [4]:
sec_metadata_final["Has Income Statement Flag"].value_counts()

True     1939
False      99
Name: Has Income Statement Flag, dtype: int64

In [10]:
sec_metadata_final.groupby("VersionId").head(2)

,DocumentId,doc cnt,VersionId,CompanyId,Company Name,documentToObjectRelId,elementGroupId,PeriodOfReport,Language,FilingDate,FilingDateSource,PeriodType,Has Income Statement Flag,Is Preliminary Flag,Is Amendment Flag,Is Errata Amendment Flag,Has Notes Flag,Has Audited Financials Flag,Category
0,322994151,4,97924048,24927252,Noranda Alumina LLC,347206249,-1295509969,2009-08-31,English,2010-05-14 16:28:42,Filing date on the website,Quarter,True,0.0,0.0,0.0,1.0,0.0,VALIDATION
1,322994151,4,97924048,24927252,Noranda Alumina LLC,347206249,-1295509965,2008-12-31,English,2010-05-14 16:28:42,Filing date on the website,Annual,True,0.0,0.0,0.0,1.0,1.0,TRAIN
4,338230196,2,338279360,34265589,Altra Industrial Motion Corp.,337512843,-1339361167,2017-12-31,English,2018-05-08 16:38:30,Filing date on the website,Annual,True,1.0,0.0,0.0,0.0,0.0,TRAIN
5,338230196,2,338279360,563559671,"Stevens Holding Company, Inc.",337582687,-1338930233,2017-12-31,English,2018-05-08 16:38:30,Filing date on the website,Annual,True,0.0,0.0,0.0,1.0,1.0,TRAIN
6,339642600,1,339642784,332752,"ADTRAN, Inc.",337686193,-1338386285,2018-03-31,English,2018-05-09 16:39:54,Filing date on the website,Quarter,True,0.0,0.0,0.0,1.0,NaN,TRAIN
7,340600968,1,340602140,431781010,"Ranger Energy Services, Inc.",337880721,-1337599935,2018-03-31,English,2018-05-10 06:51:25,Filing date on the website,Quarter,True,0.0,0.0,0.0,1.0,NaN,TRAIN
8,368641604,1,368661212,340391,"Pyxus International, Inc.",342730417,-1316038839,2018-03-31,English,2018-06-07 10:07:03,Filing date on the website,Annual,True,0.0,0.0,0.0,1.0,1.0,TRAIN
11,380436538,4,380436896,34265589,Altra Industrial Motion Corp.,345124725,-1304978009,2018-03-31,English,2018-06-21 17:09:18,Filing date on the website,Quarter,True,0.0,0.0,0.0,1.0,NaN,VALIDATION
12,380436538,4,380436896,34265589,Altra Industrial Motion Corp.,345124725,-1304747189,2017-12-31,English,2018-06-21 17:09:18,Filing date on the website,Annual,True,1.0,0.0,0.0,1.0,1.0,TRAIN
17,398045508,4,398046082,118474533,Verra Mobility Corporation,366500779,-1213353795,2018-03-31,English,2018-07-12 16:37:07,Filing date on the website,Quarter,True,0.0,0.0,0.0,1.0,0.0,VALIDATION


In [16]:
a=list(sec_metadata_final[sec_metadata_final["Has Income Statement Flag"]==False]["VersionId"].unique())

In [17]:
b=list(sec_metadata_final[sec_metadata_final["Has Income Statement Flag"]==True]["VersionId"].unique())

In [23]:
c=set(a).intersection(set(b))

In [28]:
d=sec_metadata_final[(sec_metadata_final.VersionId.isin(c))&(sec_metadata_final["Has Income Statement Flag"]==False)]
sec_metadata_final=sec_metadata.merge(d, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)

In [29]:
sec_metadata_final["Has Income Statement Flag"].value_counts()

True     1939
False    1107
Name: Has Income Statement Flag, dtype: int64

In [36]:
sec_metadata_if=sec_metadata_final[["VersionId","Has Income Statement Flag"]]

In [37]:
sec_metadata_if=sec_metadata_if[~sec_metadata_if.duplicated()]

In [38]:
sec_metadata_if[sec_metadata_if.VersionId==230292662]

,VersionId,Has Income Statement Flag
645,230292662,True


In [39]:
sec_metadata_if["Has Income Statement Flag"].value_counts()

True     1267
False     917
Name: Has Income Statement Flag, dtype: int64

In [565]:
k=0
onlyfiles = [f for f in listdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\SEC_TRAIN') if isfile(join('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\SEC_TRAIN', f))]
os.chdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\SEC_TRAIN')
mv_df_inc=pd.DataFrame(columns=(['versionid',"companyname","incomeflag"]))
for file in onlyfiles:
    lin=""
    print(file)
    a=file
    mv_df_inc.loc[k,['versionid']]=os.path.splitext(a)[0]
    with open(file,encoding="utf8") as input_data:
        for line in input_data:
            mv_df_inc.loc[k,['companyname']]=line.split(" ",1)[1].strip().split("  ",1)[0]
            break
        for line in input_data:
            if any(x in line.lower() for x in ["consolidated statements","consolidated statement","condensed","consolidated","data","statement of","statements of","statistics"]):
                if any(x in line.lower() for x in ["operations","income","financial","earnings","profit & loss","profit&loss","profit and loss"]):
                    if line.strip()[0].isupper():
                        if not any(x in line.lower() for x in ["data of","data is","on data","for data","to","others","would","should","could","by"]):
                            if not any(x in line.lower() for x in ["agreement","agreements","asset","assets","cash flow","cash flows","balance sheet","balance sheets","equity"]):
                                lin+=line
    if len(lin)==0:
        mv_df_inc.loc[k,['incomeflag']]=False
        k+=1
    else:
        mv_df_inc.loc[k,['incomeflag']]=True
        k+=1        

175605384.txt
209271852.txt
226135784.txt
226157588.txt
230288860.txt
230292662.txt
232460148.txt
236972360.txt
237114582.txt
237143018.txt
239593846.txt
242236708.txt
242607122.txt
243576070.txt
243724800.txt
244735926.txt
244842314.txt
246273214.txt
249898752.txt
250984690.txt
252567172.txt
252688048.txt
253498342.txt
253651946.txt
253715340.txt
253827726.txt
253835662.txt
253874620.txt
253891268.txt
256166326.txt
256378252.txt
256468776.txt
257505788.txt
258140358.txt
258228198.txt
258264294.txt
258370180.txt
259118202.txt
259585392.txt
259602418.txt
259720618.txt
259844898.txt
259903996.txt
260833248.txt
260845914.txt
261066672.txt
261273360.txt
261556614.txt
261655748.txt
261699460.txt
261699932.txt
261757808.txt
263058312.txt
263081942.txt
264921594.txt
267882938.txt
270663546.txt
273075380.txt
273186958.txt
275798274.txt
278984340.txt
279035452.txt
281045440.txt
282977390.txt
285534432.txt
285661030.txt
291242490.txt
291393110.txt
291449076.txt
292669118.txt
300835588.txt
301798

471338908.txt
471339086.txt
471339880.txt
471340572.txt
471340594.txt
471341822.txt
471342308.txt
471342868.txt
471342954.txt
471343010.txt
471343498.txt
471343984.txt
471346522.txt
471348592.txt
471348794.txt
471349672.txt
471349678.txt
471350510.txt
471351264.txt
471351616.txt
471352770.txt
471352954.txt
471353888.txt
471356462.txt
471358032.txt
471358164.txt
471367136.txt
471367146.txt
471367194.txt
471367494.txt
471368794.txt
471369252.txt
471370952.txt
471373380.txt
471438356.txt
471438562.txt
471439452.txt
471439642.txt
471439654.txt
471439732.txt
471439740.txt
471440742.txt
471444576.txt
471444584.txt
471444594.txt
471444786.txt
471722940.txt
471724936.txt
471725320.txt
471725516.txt
471725606.txt
471727162.txt
471727408.txt
471727630.txt
471727786.txt
471727788.txt
471729450.txt
471729850.txt
471736558.txt
471740776.txt
471775728.txt
471779360.txt
471779652.txt
471781914.txt
471785430.txt
471786042.txt
471798508.txt
471800736.txt
471800780.txt
471811774.txt
471812656.txt
471814

477746362.txt
477749356.txt
477779596.txt
477782276.txt
477784694.txt
477785082.txt
477785878.txt
477787830.txt
477797468.txt
477810762.txt
477814142.txt
477869622.txt
477887590.txt
477896794.txt
477900588.txt
477914856.txt
477943216.txt
477993898.txt
478003336.txt
478009080.txt
478016144.txt
478016784.txt
478033228.txt
478036416.txt
478050230.txt
478063634.txt
478063722.txt
478063786.txt
478066258.txt
478066522.txt
478069716.txt
478070760.txt
478072374.txt
478072532.txt
478074270.txt
478074874.txt
478079492.txt
478080154.txt
478082496.txt
478082624.txt
478083326.txt
478083578.txt
478084156.txt
478085262.txt
478086456.txt
478094168.txt
478096780.txt
478098648.txt
478098770.txt
478100790.txt
478103562.txt
478108324.txt
478110292.txt
478110710.txt
478110986.txt
478488010.txt
478489604.txt
478490330.txt
478490426.txt
478490688.txt
478490806.txt
478490932.txt
478490944.txt
478491182.txt
478493726.txt
478494108.txt
478513056.txt
478514498.txt
478528392.txt
478528404.txt
478535276.txt
478535

486435886.txt
486435922.txt
486436472.txt
486436810.txt
486436982.txt
486438976.txt
486439050.txt
486439058.txt
486439158.txt
486439180.txt
486439654.txt
486440974.txt
486442184.txt
486442406.txt
486442632.txt
486442782.txt
486442914.txt
486442922.txt
486442924.txt
486442926.txt
486443700.txt
486444292.txt
486445084.txt
486446774.txt
486448318.txt
486452780.txt
486452876.txt
486453682.txt
486453684.txt
486454458.txt
486454852.txt
486455780.txt
486455838.txt
486456024.txt
486456046.txt
487986840.txt
487986864.txt
487986890.txt
487988466.txt
487996482.txt
487997078.txt
488048120.txt
488060168.txt
488068864.txt
97924048.txt


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 1405: invalid start byte

In [566]:
mv_df_inc["versionid"]=pd.to_numeric(mv_df_inc["versionid"])

In [570]:
mv_main_inc_df=pd.merge(mv_df_inc,sec_metadata_if,left_on = ['versionid'], right_on = ['VersionId'])

In [571]:
mv_main_inc_df.head(2)

,versionid,companyname,incomeflag,VersionId,Has Income Statement Flag
0,230288860,"Hillenbrand, Inc.",True,230288860,True
1,230292662,MID AMERICA APARTMENT COMMUNITIES INC,True,230292662,True


In [572]:
mv_main_inc_df.shape

(1664, 5)

In [573]:
mv_main_inc_df.incomeflag.value_counts()

True     1275
False     388
Name: incomeflag, dtype: int64

In [574]:
mv_main_inc_df["Has Income Statement Flag"].value_counts()

True     937
False    727
Name: Has Income Statement Flag, dtype: int64

In [575]:
mv_main_inc_df[mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"]].shape

(404, 5)

In [576]:
mv_main_inc_df[mv_main_inc_df["Has Income Statement Flag"]==mv_main_inc_df["incomeflag"]].shape

(1260, 5)

In [577]:
mv_main_inc_df[(mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"])&(mv_main_inc_df["incomeflag"]==True)].shape

(371, 5)

In [578]:
mv_main_inc_df[(mv_main_inc_df["Has Income Statement Flag"]!=mv_main_inc_df["incomeflag"])&(mv_main_inc_df["incomeflag"]==False)].shape

(32, 5)

In [563]:
lin=""
with open("488068864.txt",encoding="utf8") as input_data:
    for line in input_data:
        if any(x in line.lower() for x in ["consolidated statements","consolidated statement","condensed","consolidated","data","statement of","statements of","statistics"]):
            #print("1",line)
            if any(x in line.lower() for x in ["operations","income","financial","earnings","profit & loss","profit&loss","profit and loss"]):
                #print("2",line)
                if line.strip()[0].isupper():
                    #print("3",line)
                    if not any(x in line.lower() for x in ["data of","data is","on data","for data","to","others","would","should","could","by"]):
                        #print("4",line)
                        if not any(x in line.lower() for x in ["agreement","agreements","asset","assets","cash flow","cash flows","balance sheet","balance sheets","equity"]):
                            lin+=line
    print(lin)

Key Financial and Operating Statistics



In [530]:
print(mv_main_inc_df["Has Income Statement Flag"].count())
print(mv_main_inc_df.incomeflag.count())

1664
1663


In [564]:
mv_main_inc_df[(mv_main_inc_df["Has Income Statement Flag"]==True)&(mv_main_inc_df["incomeflag"]==False)]

,versionid,companyname,incomeflag,VersionId,Has Income Statement Flag
89,428123212,"i3 Verticals, Inc.",False,428123212,True
179,467815274,STARCORE INTERNATIONAL MINES LTD.,False,467815274,True
220,468451008,VALMONT INDUSTRIES INC,False,468451008,True
399,470966076,Verso Corp,False,470966076,True
493,471722940,American Homes 4 Rent,False,471722940,True
511,471785430,DHX Media Ltd.,False,471785430,True
550,472088718,NEOGEN CORP,False,472088718,True
572,472152554,VITRO DIAGNOSTICS INC,False,472152554,True
578,472176622,"Delek US Holdings, Inc.",False,472176622,True
592,472587728,Forward Pharma A/S,False,472587728,True


In [268]:
y=sec_metadata_if[sec_metadata_if.VersionId.isin(lst)].groupby("VersionId").count()==1

In [274]:
y=sec_metadata_if[(sec_metadata_if.VersionId.isin(y[y["Has Income Statement Flag"]==False].index))&(sec_metadata_if["Has Income Statement Flag"]==False)]
sec_metadata_if=sec_metadata_if.merge(y, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)

In [253]:
sec_metadata_if[sec_metadata_if.VersionId.isin(y)&(sec_metadata_if["Has Income Statement Flag"]==False)]

,VersionId,Has Income Statement Flag
19,398046082,False
35,444879438,False
40,447617684,False
45,454695766,False
49,465536652,False
58,476747374,False
61,477205816,False
73,480894374,False
85,483648664,False
93,487997078,False


In [220]:
sec_metadata_if[sec_metadata_if.VersionId==338885212]

,VersionId,Has Income Statement Flag
682,338885212,True
685,338885212,False


In [218]:
x

,Has Income Statement Flag
VersionId,
273075380,True
338885212,True
342574798,True
350666530,True
383711294,True
398046082,True
426757198,True
444879438,True
447617684,True
